In [ ]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as pl
from mpl_toolkits.mplot3d.axes3d import Axes3D
from PIL import Image
import functions
from GP import GP
from GP_grad import GP_grad
import time
import imageio

In [ ]:
Noise = True # Specify if noisy or not
Noise_level = 0.01 # if noisy, what is the noise sd
'''If use the scaled X and Y, rememeber to scale the noise as well'''

In [ ]:
objective = functions.sincos(Noise, Noise_level)
# objective = functions.sin(Noise, Noise_level)

bounds = np.array([item[1] for item in sorted(objective.bounds.items(), key=lambda x: x[0])], dtype=np.float64)
print(bounds) 

f = objective.func
print(f( np.array([2, 1]) ))
# np.sin(np.array([2, 1]))

dim = objective.input_dim
print('dim: ', dim)

In [ ]:
# max budget for finding good point
Budget = 4
# set seed
seed = 0
# Acq_names=['PI','PG','EI','EG','gp_ucb','random','TS','SATS','G_ES','MES']
acquation_function = 'random'

In [ ]:
def draw(n_random_draws=3):
    """ samples some points"""
    # np.random.seed(seed) 
    # should draw randomly
    X = np.random.uniform(bounds[:, 0], bounds[:, 1], size=(n_random_draws, dim))
    return X, f(X)

gp = GP_grad(bounds, Noise, Noise_level)

def get_kernel(sample_size=4):
    np.random.seed(seed)
    X = np.asarray(np.random.uniform(bounds[:, 0], bounds[:, 1], size=(sample_size, dim)))
    gp.set_data(X, f(X))
    # gp.optimize()
    gp.fit()
    return gp.get_hyper()

In [ ]:
# creating meshgrid to plot over entire range
x1 = np.linspace(2, 10, 100)
x2 = np.linspace(2, 10, 100)

X1, X2 = np.meshgrid(x1, x2)
mesh = np.vstack((X1.flatten(), X2.flatten())).T

In [ ]:
get_kernel()
n = 50

# posterior mean and cov
mu, covar = gp.posterior(mesh)
s = np.sqrt(np.diag(covar))
mu = np.squeeze(mu)

Kt = gp.cov_RBF(mesh, mesh, gp.get_hyper())
K_11 = gp.K11(mesh)
K_01 = gp.K01(mesh)

In [ ]:
gp.X.shape, gp.y.shape

In [ ]:
K_11.shape, K_01.shape

In [ ]:
# posterior of derivative
# partial x1
gp.set_p(0)
mu_grad_1, covar_grad_1 = gp.posterior_grad(mesh)
s_grad_1 = np.sqrt(np.diag(covar_grad_1))
mu_grad_1 = np.squeeze(mu_grad_1)

# partial x2
gp.set_p(1)
mu_grad_2, covar_grad_2 = gp.posterior_grad(mesh)
s_grad_2 = np.sqrt(np.diag(covar_grad_2))
mu_grad_2 = np.squeeze(mu_grad_2)

In [ ]:
mu_grad_1.shape, covar_grad_1.shape

In [ ]:

fig, ((ax1, ax2)) = pl.subplots(
    nrows=1, ncols=2, figsize=(14, 6), dpi=100
)

im = ax2.pcolormesh(X1, X2, s.reshape(X1.shape), cmap="jet")
fig.colorbar(im, ax=ax2)

im1 = ax1.contour(X1, X2, (np.sin(mesh[:,0])*np.cos(mesh[:,1])/np.sqrt(mesh[:,0]*mesh[:,1])).reshape(X1.shape), cmap="PuBuGn")
im2 = ax1.contour(X1, X2, mu.reshape(X1.shape), cmap="YlOrRd")

fig.colorbar(im1, ax=ax1)
fig.colorbar(im2, ax=ax1)

ax1.plot(gp.X[:, 0], gp.X[:, 1], "ok", markersize=5, alpha=0.8)
ax1.title.set_text("Contour of Noisy sin(x)cos(y)/sqrt(xy)")
ax2.title.set_text("Standard deviation")

In [ ]:
x = mesh[:,0]
y = mesh[:,1]
p1 = y*np.cos(y)* ( (2*x*np.cos(x)) - np.sin(x) ) / (2* (x*y)**(3/2))
p2 = - x*np.sin(x)* ( (2*y*np.sin(y)) + np.cos(y) ) / (2* (x*y)**(3/2))

# out_0 = (np.sin(t[:, 1]) * (2 * t[:, 0] * np.cos(t[:, 0]) - np.sin(t[:, 0]))) / (
#     2 * t[:, 0] * np.sqrt(t[:, 0]) * np.sqrt(t[:, 1])
# )
# out_1 = (np.sin(t[:, 0]) * (2 * t[:, 1] * np.cos(t[:, 1]) - np.sin(t[:, 1]))) / (
#     2 * t[:, 1] * np.sqrt(t[:, 0]) * np.sqrt(t[:, 1])
# )
fig, ((ax1, ax2), (ax3, ax4)) = pl.subplots(
    nrows=2, ncols=2, figsize=(14, 9), dpi=100
)

im = ax2.pcolormesh(X1, X2, s_grad_1.reshape(X1.shape), cmap="jet")
fig.colorbar(im, ax=ax2)
im1 = ax1.contour(X1, X2, p1.reshape(X1.shape), cmap="PuBuGn")
im2 = ax1.contour(X1, X2, mu_grad_1.reshape(X1.shape), cmap="YlOrRd")

fig.colorbar(im1, ax=ax1)
fig.colorbar(im2, ax=ax1)

ax1.plot(gp.X[:, 0], gp.X[:, 1], "ok", markersize=5, alpha=0.8)
ax1.title.set_text("Mean of Posterior Pratial x")
ax2.title.set_text("Std of Posterior Pratial x")

im3 = ax4.pcolormesh(X1, X2, s_grad_2.reshape(X1.shape), cmap="jet")
fig.colorbar(im3, ax=ax4)
im4 = ax3.contour(X1, X2, p2.reshape(X1.shape), cmap="PuBuGn")
im5 = ax3.contour(X1, X2, mu_grad_2.reshape(X1.shape), cmap="YlOrRd")

fig.colorbar(im4, ax=ax3)
fig.colorbar(im5, ax=ax3)

ax3.plot(gp.X[:, 0], gp.X[:, 1], "ok", markersize=5, alpha=0.8)
ax3.title.set_text("Mean of Posterior Pratial y")
ax4.title.set_text("Std of Posterior Pratial y")

pl.show()

In [ ]:
gp = GP_grad(bounds, Noise, Noise_level)
Budget = 10
np.random.seed(seed)  

X = np.asarray(np.random.uniform(bounds[:, 0], bounds[:, 1], size=(4, dim)))
gp.set_data(X, f(X))
gp.fit()
mu, covar = gp.posterior(mesh)

for i in range(1,  Budget):
    # posterior of GP given gp.X, gp.y
    x = mesh[:,0]
    y = mesh[:,1]

    mu, covar = gp.posterior(mesh)
    s = np.sqrt(np.diag(covar))
    mu = np.squeeze(mu)

    # posterior of GP derivative given gp.X, gp.y
    gp.set_p(0)
    mu_grad_1, covar_grad_1 = gp.posterior_grad(mesh)
    s_grad_1 = np.sqrt(np.diag(covar_grad_1))
    mu_grad_1 = np.squeeze(mu_grad_1)

    # partial x2
    gp.set_p(1)
    mu_grad_2, covar_grad_2 = gp.posterior_grad(mesh)
    s_grad_2 = np.sqrt(np.diag(covar_grad_2))
    mu_grad_2 = np.squeeze(mu_grad_2)

    fig, ((ax1, ax2), (ax3, ax4), (ax5, ax6)) = pl.subplots(3, 2, figsize=(20, 12))

    # plot function
    im = ax2.pcolormesh(X1, X2, s.reshape(X1.shape), cmap="jet")
    fig.colorbar(im, ax=ax2)

    im1 = ax1.contour(X1, X2, (np.sin(x)*np.cos(y)/np.sqrt(x*y)).reshape(X1.shape), cmap="PuBuGn")
    im2 = ax1.contour(X1, X2, mu.reshape(X1.shape), cmap="YlOrRd")

    fig.colorbar(im1, ax=ax1)
    fig.colorbar(im2, ax=ax1)

    ax1.plot(gp.X[:, 0], gp.X[:, 1], "ok", markersize=5, alpha=0.8)
    ax1.title.set_text("Contour of Noisy sin(x)cos(y)/sqrt(xy)")
    ax2.title.set_text("Standard deviation")

    # plot derivative
    p1 = y*np.cos(y)* ( (2*x*np.cos(x)) - np.sin(x) ) / (2* (x*y)**(3/2))
    p2 = - x*np.sin(x)* ( (2*y*np.sin(y)) + np.cos(y) ) / (2* (x*y)**(3/2))

    im = ax4.pcolormesh(X1, X2, s_grad_1.reshape(X1.shape), cmap="jet")
    fig.colorbar(im, ax=ax4)
    im1 = ax3.contour(X1, X2, p1.reshape(X1.shape), cmap="PuBuGn")
    im2 = ax3.contour(X1, X2, mu_grad_1.reshape(X1.shape), cmap="YlOrRd")

    fig.colorbar(im1, ax=ax3)
    fig.colorbar(im2, ax=ax3)

    ax3.plot(gp.X[:, 0], gp.X[:, 1], "ok", markersize=5, alpha=0.8)
    ax3.title.set_text("Mean of Posterior Pratial x")
    ax4.title.set_text("Std of Posterior Pratial x")

    im3 = ax6.pcolormesh(X1, X2, s_grad_2.reshape(X1.shape), cmap="jet")
    fig.colorbar(im3, ax=ax6)
    im4 = ax5.contour(X1, X2, p2.reshape(X1.shape), cmap="PuBuGn")
    im5 = ax5.contour(X1, X2, mu_grad_2.reshape(X1.shape), cmap="YlOrRd")

    fig.colorbar(im4, ax=ax5)
    fig.colorbar(im5, ax=ax5)

    ax5.plot(gp.X[:, 0], gp.X[:, 1], "ok", markersize=5, alpha=0.8)
    ax5.title.set_text("Mean of Posterior Pratial y")
    ax6.title.set_text("Std of Posterior Pratial y")

    pl.show()

    # sample a new point
    X, f_X = draw(1)
    gp.add_data(X, f_X)
    gp.fit()

In [ ]:
f(np.random.uniform(1, 10, size=(1, 2))).shape

In [ ]:
# Creates the .gif from the saved plots for the 2D_Plots dir.
filenames = []
for i in range(1, 127):
    filename = 'sin_' + str(i) + '_' + noise +' .png'
    filenames.append(filename)

frames=[]
for filename in filenames:
    frames.append(imageio.imread('Result_Plots/' + filename))
exportname = 'sin_' + noise + '.gif'

kargs = { 'duration': 0.5 }
imageio.mimsave(exportname, frames, 'GIF', **kargs)